# Markt Agent

In [ ]:
# initialise

!pip install ccxt
!pip install ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.1/641.1 kB 24.9 MB/s eta 0:00:00


In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=92a95aed86b993ae05d327c861cfff4f7e301aa0ac9d936a17cf24b35d2f336f
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


## ML Training Code

### v1.0

In [ ]:
import ccxt
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import ta

# === CONFIGURATION ===
coins = ["DOGE/USDT", "SHIB/USDT", "PEPE/USDT", "BONK/USDT"]
timeframe = '5m'
limit = 3000  # ca. 10-12 Tage bei 5-min Kerzen

exchange = ccxt.kucoin()  # KuCoin Exchange

all_data = []

for coin in coins:
    try:
        ohlcv = exchange.fetch_ohlcv(coin, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['time','open','high','low','close','volume'])
        df['time'] = pd.to_datetime(df['time'], unit='ms')

        # === Features ===
        df['close_change_pct'] = df['close'].pct_change()
        df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
        df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
        df['ma20_diff'] = (df['close'] - df['close'].rolling(20).mean()) / df['close'].rolling(20).mean()
        df['ma50_diff'] = (df['close'] - df['close'].rolling(50).mean()) / df['close'].rolling(50).mean()
        df['momentum'] = df['close'] - df['close'].shift(3)
        df['vol_change'] = df['volume'].pct_change()

        # === Bereinigung: NaN / Inf / Extreme Werte ===
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)

        # Extremwerte clippen
        for col in ['close_change_pct','ma20_diff','ma50_diff','momentum','vol_change']:
            df[col] = df[col].clip(lower=-10, upper=10)

        # === Target definieren ===
        df['target'] = ((df['close'].shift(-3) - df['close']) / df['close']) > 0.002  # 0.2% Anstieg in 3 Kerzen
        df['target'] = df['target'].astype(int)

        df['coin'] = coin
        all_data.append(df)

    except Exception as e:
        print(f"Error loading {coin}: {e}")

# === Combine all coins ===
data = pd.concat(all_data)

features = ['close_change_pct','rsi','atr','ma20_diff','ma50_diff','momentum','vol_change']
X = data[features]
y = data['target']

# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# === XGBoost Model Training ===
model = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train)

# === Test Accuracy ===
pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))

# === Save Model ===
joblib.dump(model, "ml_model_meme_multi_kucoin.pkl")
print("✅ Multi-Coin ML Model gespeichert als 'ml_model_meme_multi_kucoin.pkl'")


Accuracy: 0.6885245901639344
✅ Multi-Coin ML Model gespeichert als 'ml_model_meme_multi_kucoin.pkl'


### v2.0

In [ ]:
import ccxt
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import ta

# === CONFIGURATION ===
coins = ["DOGE/USDT", "SHIB/USDT", "PEPE/USDT", "BONK/USDT"]
timeframe = '5m'
limit = 25920  # mehr Daten für besseres Training

exchange = ccxt.kucoin()  # KuCoin Exchange

all_data = []

for coin in coins:
    try:
        ohlcv = exchange.fetch_ohlcv(coin, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['time','open','high','low','close','volume'])
        df['time'] = pd.to_datetime(df['time'], unit='ms')

        # === Features ===
        df['close_change_pct'] = df['close'].pct_change()
        df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
        df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
        df['ma20_diff'] = (df['close'] - df['close'].rolling(20).mean()) / df['close'].rolling(20).mean()
        df['ma50_diff'] = (df['close'] - df['close'].rolling(50).mean()) / df['close'].rolling(50).mean()
        df['momentum'] = df['close'] - df['close'].shift(3)
        df['vol_change'] = df['volume'].pct_change()
        df['vol_ma5'] = df['volume'].rolling(5).mean()
        df['vol_ma10'] = df['volume'].rolling(10).mean()
        df['vol_ratio'] = df['volume'] / (df['vol_ma10']+1e-8)  # Verhältnis aktuelles Volumen / 10er-MA

        # Candle-Muster (Bullish/Bearish Engulfing)
        df['bull_engulf'] = ((df['close'] > df['open'].shift(1)) & (df['open'] < df['close'].shift(1))).astype(int)
        df['bear_engulf'] = ((df['close'] < df['open'].shift(1)) & (df['open'] > df['close'].shift(1))).astype(int)

        # === Bereinigung ===
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        for col in ['close_change_pct','ma20_diff','ma50_diff','momentum','vol_change','vol_ratio']:
            df[col] = df[col].clip(lower=-10, upper=10)

        # === Target: ATR-basiert ===
        df['target'] = ((df['close'].shift(-3) - df['close']) / df['atr']) > 0.5
        df['target'] = df['target'].astype(int)

        df['coin'] = coin
        all_data.append(df)

    except Exception as e:
        print(f"Error loading {coin}: {e}")

# === Combine all coins ===
data = pd.concat(all_data)

features = ['close_change_pct','rsi','atr','ma20_diff','ma50_diff','momentum',
            'vol_change','vol_ma5','vol_ma10','vol_ratio','bull_engulf','bear_engulf']
X = data[features]
y = data['target']

# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# === XGBoost Training (Hyperparameter-optimiert) ===
model = XGBClassifier(n_estimators=500, max_depth=8, learning_rate=0.03, subsample=0.8, colsample_bytree=0.8, random_state=42)
model.fit(X_train, y_train)

# === Test Accuracy ===
pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))

# === Save Model ===
joblib.dump(model, "ml_model_meme_multi_kucoin_v2.pkl")
print("✅ Verbesserte Multi-Coin ML Model gespeichert als 'ml_model_meme_multi_kucoin_v2.pkl'")


Accuracy: 0.6583261432269197
✅ Verbesserte Multi-Coin ML Model gespeichert als 'ml_model_meme_multi_kucoin_v2.pkl'


## Bot

In [ ]:
import ccxt
import pandas as pd
import numpy as np
import ta
import joblib

# === CONFIGURATION ===
coins = ["DOGE/USDT", "SHIB/USDT", "PEPE/USDT", "BONK/USDT"]
start_balance = 100
interval = '5m'
limit = 25920  # kürzere Testperiode

ml_model_path = "ml_model_meme_multi_kucoin_v2.pkl"
ml_threshold = 0.6
rsi_buy = 30
rsi_sell = 70

exchange = ccxt.kucoin()
exchange.load_markets()
model = joblib.load(ml_model_path)

results = []
all_trades = []

available_coins = [c for c in coins if c in exchange.markets]
print("Verfügbare Coins für Backtest:", available_coins)

for coin in available_coins:
    try:
        ohlcv = exchange.fetch_ohlcv(coin, interval, limit=limit)
        if len(ohlcv) == 0:
            print(f"No data for {coin}, skipping...")
            continue

        df = pd.DataFrame(ohlcv, columns=['time','open','high','low','close','volume'])
        df['time'] = pd.to_datetime(df['time'], unit='ms')

        # === Features berechnen ===
        df['close_change_pct'] = df['close'].pct_change()
        df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=7).rsi()  # kürzeres RSI
        df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=7).average_true_range()  # kürzeres ATR
        df['ma20_diff'] = (df['close'] - df['close'].rolling(10).mean()) / df['close'].rolling(10).mean()  # kürzeres MA
        df['ma50_diff'] = (df['close'] - df['close'].rolling(20).mean()) / df['close'].rolling(20).mean()  # kürzeres MA
        df['momentum'] = df['close'] - df['close'].shift(3)
        df['vol_change'] = df['volume'].pct_change()
        df['vol_ma5'] = df['volume'].rolling(3).mean()
        df['vol_ma10'] = df['volume'].rolling(5).mean()
        df['vol_ratio'] = df['volume'] / (df['vol_ma10'] + 1e-8)
        df['bull_engulf'] = ((df['close'] > df['open'].shift(1)) & (df['open'] < df['close'].shift(1))).astype(int)
        df['bear_engulf'] = ((df['close'] < df['open'].shift(1)) & (df['open'] > df['close'].shift(1))).astype(int)

        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        features = ['close_change_pct','rsi','atr','ma20_diff','ma50_diff','momentum',
                    'vol_change','vol_ma5','vol_ma10','vol_ratio','bull_engulf','bear_engulf']

        # NaNs sanft füllen
        df[features] = df[features].fillna(method='bfill').fillna(method='ffill')

        balance = start_balance
        position = None
        entry_price = 0
        trades = []

        for i in range(len(df)):
            price = df.loc[i,'close']
            rsi = df.loc[i,'rsi']
            X_current = df.loc[i, features].values.reshape(1,-1)
            ml_score = model.predict_proba(X_current)[0][1]

            # Kaufbedingung
            if position is None and rsi < rsi_buy and ml_score > ml_threshold:
                position = 'LONG'
                entry_price = price
                trades.append({'coin':coin,'time':df.loc[i,'time'],'action':'BUY','price':price,'rsi':rsi,'ml_score':ml_score})

            # Verkauf
            elif position == 'LONG' and (rsi > rsi_sell or ml_score < (1-ml_threshold)):
                profit_pct = (price - entry_price)/entry_price*100
                balance *= (1 + profit_pct/100)
                position = None
                trades.append({'coin':coin,'time':df.loc[i,'time'],'action':'SELL','price':price,'rsi':rsi,'ml_score':ml_score,'profit_%':profit_pct})

        results.append({'coin':coin,'final_balance':balance,'change_%':(balance-start_balance)/start_balance*100})
        all_trades.extend(trades)

    except Exception as e:
        print(f"Error with {coin}: {e}")

# === Export CSV ===
results_df = pd.DataFrame(results)
trades_df = pd.DataFrame(all_trades)
results_df.to_csv("backtest_summary_ml_meme.csv", index=False)
trades_df.to_csv("backtest_trades_ml_meme.csv", index=False)

print("✅ ML+RSI Backtest für Meme-Coins abgeschlossen!")
print(results_df)


Verfügbare Coins für Backtest: ['DOGE/USDT', 'SHIB/USDT', 'PEPE/USDT', 'BONK/USDT']


/tmp/ipython-input-3506041094.py:58: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[features] = df[features].fillna(method='bfill').fillna(method='ffill')
/tmp/ipython-input-3506041094.py:58: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[features] = df[features].fillna(method='bfill').fillna(method='ffill')
/tmp/ipython-input-3506041094.py:58: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[features] = df[features].fillna(method='bfill').fillna(method='ffill')
/tmp/ipython-input-3506041094.py:58: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[features] = df[features].fillna(method='bfill').fillna(method='ffill')


✅ ML+RSI Backtest für Meme-Coins abgeschlossen!
        coin  final_balance   change_%
0  DOGE/USDT     108.056176   8.056176
1  SHIB/USDT     107.038974   7.038974
2  PEPE/USDT     111.991882  11.991882
3  BONK/USDT     104.390560   4.390560
